In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
# Thiết lập tùy chọn hiển thị của pandas
pd.set_option('display.max_columns', None)  # Hiển thị tất cả các cột
pd.set_option('display.max_rows', None)     # Hiển thị tất cả các hàng

# Đặt lại tùy chọn hiển thị về giá trị mặc định nếu cần
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_rows')

In [3]:
def check_missing_data(df, columns):
    """
    Kiểm tra dữ liệu bị thiếu trong các cột được chỉ định của DataFrame.
    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        columns (list): Danh sách tên các cột cần kiểm tra.
    Returns:
        DataFrame: DataFrame chứa các dòng dữ liệu có dữ liệu bị thiếu trong các cột được chỉ định.
    """
    missing_data = df[df[columns].isnull().any(axis=1)]
    if missing_data.empty:
        print("Không có dữ liệu bị thiếu trong các cột:", ", ".join(columns))
    else:
        print("Số lượng dòng dữ liệu bị thiếu trong các cột", ", ".join(columns), "là:", missing_data.shape[0])
        print("Các dòng dữ liệu bị thiếu trong các cột", ", ".join(columns), "là:")
        # print(missing_data)
    return missing_data

def check_duplicate_data(df):
    """
    Hàm để kiểm tra và theo dõi dữ liệu trùng lặp trong DataFrame.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.

    Returns:
        DataFrame: DataFrame chứa các dòng dữ liệu trùng lặp.
    """
    # Kiểm tra dữ liệu trùng lặp
    duplicate_data = df[df.duplicated()]

    # In thông báo nếu không có dữ liệu trùng lặp
    if duplicate_data.empty:
        print("Không có dữ liệu trùng lặp.")
    else:
      print("Những dữ liệu trùng lặp: ", duplicate_data)
    return duplicate_data


def count_date_formats(df, column, print_result=False):
    """
    Hàm để kiểm tra và theo dõi số lượng định dạng của dữ liệu ngày tháng trong một cột.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        column (str): Tên của cột cần kiểm tra.
        print_result (bool, optional): True nếu muốn in kết quả ra màn hình, False nếu không (mặc định là False).

    Returns:
        dict: Một từ điển với các định dạng ngày tháng làm khóa và số lượng dòng có mỗi định dạng làm giá trị.
    """
    # Chuyển đổi các giá trị của cột thành chuỗi định dạng ngày tháng
    date_strings = df[column].apply(lambda x: x.strftime('%d/%m/%Y') if isinstance(x, pd.Timestamp) else x)

    # Đếm số lượng giá trị duy nhất cho mỗi định dạng ngày tháng
    format_counts = date_strings.value_counts()

    # Chuyển đổi kết quả thành từ điển
    format_counts_dict = format_counts.to_dict()

    # In kết quả nếu cần
    if print_result:
        print("Số lượng định dạng của dữ liệu ngày tháng:")
        for format, count in format_counts_dict.items():
            print(f"{format}: {count} dòng")

    return format_counts_dict

def score_labeling(df, column_name, bins=[0, 4, 5, 6, 7, 8, 9, 10], labels=['kém', 'yếu', 'trung bình', 'trung bình khá', 'khá', 'giỏi', 'xuất sắc']):
    """
    Hàm để chuyển điểm thành biến phân loại kết quả học tập và trả về cột mới.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        column_name (str): Tên cột chứa điểm cần phân loại.
        bins (list): Danh sách khoảng giá trị cho biến phân loại. Mặc định là [0, 4, 5, 6, 7, 8, 9, 10].
        labels (list): Danh sách nhãn cho các khoảng giá trị. Mặc định là ['kém', 'yếu', 'trung bình', 'trung bình khá', 'khá', 'giỏi', 'xuất sắc'].

    Returns:
        Series: Cột mới chứa biến phân loại.
    """
    # Thêm cột mới vào DataFrame chứa biến phân loại
    df['{}_xl'.format(column_name)] = pd.cut(df[column_name], bins=bins, labels=labels, right=False)
    return df['{}_xl'.format(column_name)]

## Data Quality Rules

In [17]:
# Hàm đánh giá tính đầy đủ (Completeness)
def check_completeness(row, required_columns):
    missing_count = row[required_columns].isna().sum()
    return 1 if missing_count == 0 else 0

# Hàm đánh giá tính nhất quán (Consistency)
def check_consistency(row):
    try:
        # Kiểm tra định dạng ngày tháng
        pd.to_datetime(row['ngaysinh'])
        # Kiểm tra các giá trị hợp lệ của giới tính
        if row['gioitinh'] not in ['Nam', 'Nữ']:
            return 0

        return 1
    except:
        return 0

# Hàm đánh giá tính kịp thời (Timeliness)
def check_timeliness(row, current_year):
    try:
        # Giả sử dữ liệu cập nhật mỗi năm và năm học trong dữ liệu không quá cũ
        year_of_birth = pd.to_datetime(row['ngaysinh']).year
        if year_of_birth <= current_year:
            return 1
        else:
            return 0
    except:
        return 0

# Hàm đánh giá tính hợp lệ (Validity)
def check_validity(row):
    try:
        # Kiểm tra các giá trị hợp lệ của giới tính
        if row['gioitinh'] not in ['Nam', 'Nữ']:
            return 0
        
        # Kiểm tra định dạng ngày tháng
        pd.to_datetime(row['ngaysinh'])
        
        return 1
    except:
        return 0

# Hàm đánh giá tính duy nhất (Uniqueness)
def check_uniqueness(df, column):
    duplicate_count = df.duplicated(subset=[column]).sum()
    return 1 if duplicate_count == 0 else 0

# Giả sử dssinhvien là DataFrame của bạn
# current_year là năm hiện tại
current_year = 2023

## Connect to dataset: dssinhvien_Thu.xlsx

In [5]:
# Loading the New Customer Data from the excel file
df = pd.read_excel('data/dssinhvien_Thu.xlsx')

In [6]:
# Checking first 5 records from Data
df.head(5)

,STT,masv,ngaysinh,gioitinh,noisinh,lopsh,khoa,hedt,nganh,tinhtrang,dantoc,tongiao,xuatthan,doituong,doituong_chitiet,khuvuc,vungmien,ngayvaodoan,ngayvaodang,diachi_phuongxa,diachi_quanhuyen,diachi_tinhtp,diachi_tamtru,diachi_tamtru_quanhuyen,diachi_tamtru_tinhtp,thanhtich_khenthuong,kyluat,quatrinh_congtac,chucvu_caonhat,bhyt,baoho_hoten,baoho_diachi,ghichu,baotin_diachi,bietdentu,bietdentu_other,lydohoc,lydohoc_other
0,1,6520001,05/10/1988,Nam,Sông Bé,TNTH0002,KHMT,CQUI,D480101,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,15/03/2004,NaN,NaN,Thị xã Bình Long,Tỉnh Bình Phước,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,6520002,22/10/1988,Nữ,Tỉnh Bến Tre,TNTH0002,HTTT,CQUI,D480104,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,19/05/2004,NaN,Phường 5,Thành phố Bến Tre,Tỉnh Bến Tre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,6520004,27/11/1987,Nữ,Thành phố Hồ Chí Minh,TNTH0002,HTTT,CQUI,D480104,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,11/01/2003,NaN,Xã An Ninh Tây,Huyện Đức Hòa,Tỉnh Long An,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,6520005,08/09/1988,Nam,Tỉnh Bình Thuận,TNTH0002,KHMT,CQUI,D480101,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,23/03/2003,NaN,Phường Lạc Đạo,Thành phố Phan Thiết,Tỉnh Bình Thuận,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,6520006,21/01/1988,Nam,Tỉnh Đồng Nai,TNTH0002,KHMT,CQUI,D480101,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,26/03/2003,NaN,Phường Xuân An,Thành phố Long Khánh,Tỉnh Đồng Nai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17925 entries, 0 to 17924
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   STT                      17925 non-null  int64 
 1   masv                     17925 non-null  int64 
 2   ngaysinh                 17900 non-null  object
 3   gioitinh                 17925 non-null  object
 4   noisinh                  17437 non-null  object
 5   lopsh                    17925 non-null  object
 6   khoa                     17925 non-null  object
 7   hedt                     17925 non-null  object
 8   nganh                    17925 non-null  object
 9   tinhtrang                17925 non-null  object
 10  dantoc                   17917 non-null  object
 11  tongiao                  17917 non-null  object
 12  xuatthan                 16358 non-null  object
 13  doituong                 635 non-null    object
 14  doituong_chitiet         524 non-null 

In [10]:
# Kiểm tra dữ liệu trùng lặp trong một cột cụ thể
column_name = 'masv'  # Thay 'ten_cot' bằng tên cột bạn muốn kiểm tra

duplicate_values = df[df.duplicated(subset=[column_name], keep=False)]
if duplicate_values.empty:
    print("Không có dữ liệu trùng lặp trong cột", column_name)
else:
    print("Dữ liệu trùng lặp trong cột", column_name, "là:")
    print(duplicate_values)

Không có dữ liệu trùng lặp trong cột masv


In [11]:
# Chuyển đổi dữ liệu ngày sinh từ kiểu object sang kiểu datetime
df['ngaysinh'] = pd.to_datetime(df['ngaysinh'], format='mixed', dayfirst=True)

In [12]:
# Kiểm tra định dạng của dữ liệu ngày sinh
df['ngaysinh'] = pd.to_datetime(df['ngaysinh'], format='%d/%m/%Y', errors='coerce')

# Kiểm tra xem có dữ liệu nào không phù hợp với định dạng không
invalid_dates = df[df['ngaysinh'].isna()]
if invalid_dates.empty:
    print("Tất cả các dữ liệu ngày sinh đều có đúng định dạng '%d/%m/%Y'.")
else:
    print("Các dữ liệu ngày sinh sau không phù hợp với định dạng '%d/%m/%Y':")
    print(invalid_dates.shape)
    print(invalid_dates)

Các dữ liệu ngày sinh sau không phù hợp với định dạng '%d/%m/%Y':
(25, 38)
       STT      masv ngaysinh gioitinh          noisinh     lopsh    khoa  \
615    616   7520090      NaT      Nam              NaN  TNTH0002  MMT&TT   
671    672   7520149      NaT      Nam   Tỉnh Khánh Hòa  TNTH0002    HTTT   
832    833   7520318      NaT       Nữ              NaN  TNTH0002    HTTT   
1088  1089   8520007      NaT       Nữ              NaN  TNTH0002    HTTT   
1123  1124   8520044      NaT      Nam  Tỉnh Quảng Ngãi  TNTH0002    CNPM   
1142  1143   8520063      NaT       Nữ              NaN  TNTH0002    HTTT   
1296  1297   8520237      NaT       Nữ              NaN  TNTH0002    HTTT   
1347  1348   8520299      NaT       Nữ              NaN  TNTH0002    HTTT   
1388  1389   8520342      NaT       Nữ              NaN  TNTH0002    HTTT   
1389  1390   8520343      NaT       Nữ              NaN  TNTH0002    HTTT   
1455  1456   8520410      NaT       Nữ              NaN  TNTH0002    HTTT   
1

In [14]:
df['nienkhoa'] = (df['masv'].astype(str).str[:-6]).astype(int) + 2000

# Di chuyển cột nienkhoa lên trước cột lopsh
cols = list(df.columns)
cols.insert(cols.index('lopsh'), cols.pop(cols.index('nienkhoa')))
df = df[cols]

C:\Users\levan\AppData\Local\Temp\ipykernel_2216\242881197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nienkhoa'] = (df['masv'].astype(str).str[:-6]).astype(int) + 2000


In [15]:
desired_columns = ['masv','nienkhoa','ngaysinh', 'gioitinh', 'lopsh','khoa','hedt','nganh','diachi_tinhtp']
df = df[desired_columns]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17925 entries, 0 to 17924
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   masv           17925 non-null  int64         
 1   nienkhoa       17925 non-null  int32         
 2   ngaysinh       17900 non-null  datetime64[ns]
 3   gioitinh       17925 non-null  object        
 4   lopsh          17925 non-null  object        
 5   khoa           17925 non-null  object        
 6   hedt           17925 non-null  object        
 7   nganh          17925 non-null  object        
 8   diachi_tinhtp  17767 non-null  object        
dtypes: datetime64[ns](1), int32(1), int64(1), object(6)
memory usage: 1.2+ MB


## Data Quality Assessment

In [22]:
# current_year là năm hiện tại
current_year = 2023

# Chọn các cột cần thiết để kiểm tra tính đầy đủ
required_columns = ['masv', 'ngaysinh', 'gioitinh', 'lopsh', 'khoa', 'hedt', 'nganh']

# Áp dụng các hàm đánh giá cho từng dòng
df['completeness'] = df.apply(check_completeness, axis=1, required_columns=required_columns)
df['consistency'] = df.apply(check_consistency, axis=1)
df['timeliness'] = df.apply(check_timeliness, axis=1, current_year=current_year)
df['validity'] = df.apply(check_validity, axis=1)

In [28]:
attributes = ['masv']

# Kiểm tra tính duy nhất của các dòng
df['uniqueness'] = df.duplicated(subset=attributes, keep=False).astype(int)
df['uniqueness'] = 1 - df['uniqueness']  # Đảo ngược giá trị để 1 là duy nhất và 0 là trùng lặp


In [29]:
# Tính điểm trung bình cho từng dimension
completeness_score = df['completeness'].mean()
consistency_score = df['consistency'].mean()
timeliness_score = df['timeliness'].mean()
validity_score = df['validity'].mean()

# Đánh giá tính duy nhất cho toàn bộ DataFrame
uniqueness_score = check_uniqueness(df)

In [32]:
# Danh sách các dimension và trọng số tương ứng
dimensions = ['completeness', 'consistency', 'timeliness', 'validity', 'uniqueness']
weights = {'completeness': 0.2, 'consistency': 0.2, 'timeliness': 0.2, 'validity': 0.2, 'uniqueness': 0.2}

# Tính điểm chất lượng dữ liệu cho mỗi dòng
df['data_quality_score'] = df.apply(lambda row: np.average([row[dim] for dim in dimensions], weights=[weights[dim] for dim in dimensions]), axis=1)

# Tính điểm chất lượng dữ liệu cho toàn bộ tệp dữ liệu
overall_data_quality_score = np.mean(df['data_quality_score'])

In [33]:
print(overall_data_quality_score)

0.999442119944212


In [30]:
quality_scores = pd.DataFrame({
  'Total Rows': [df.shape[0]],
  'Total Features': [df.shape[1]],
  'Completeness (%)': [completeness_score*100],
  'Consitency (%)': [consistency_score*100],
  'Validity (%)': [validity_score*100],
  'Uniqueness (%)': [uniqueness_score*100],
  'Timeliness (%)': [timeliness_score*100]
})

print(quality_scores)

   Total Rows  Total Features  Completeness (%)  Consitency (%)  Validity (%)  \
0       17925              14          99.86053           100.0         100.0   

   Uniqueness (%)  Timeliness (%)  
0           100.0        99.86053  


In [31]:
# Lưu kết quả vào file CSV
quality_scores.to_csv('report/dssinhvien_DQ_score.csv', index=False)
df.to_csv('report/dssinhvien_scored.csv', index=False)